In [1]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc
from pathlib import Path

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
import utils.path as path
from utils.VOCSegmentation import VOCSegmentation

C:\Users\Nicol\miniconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Nicol\miniconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
FB_threshold = 0.9

In [3]:
root_path = path.goback_from_current_dir(0)

json_path = root_path + 'json\\'

output_path  = root_path + r'sessions\3 Class cam\threshold_0' + str(int(FB_threshold * 10)) + '\\'

In [4]:
Path(output_path).mkdir(parents=True, exist_ok=True)

In [5]:
camnet = cam.Cam()

In [6]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Class')

In [7]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 0))

In [8]:
annotations = json.open_json(json_path + "voc-class-annotations")
N = len(annotations)
annotations = iter(annotations.items())

classes = json.open_json(json_path + "voc-classes")

In [9]:
IoU  = []
Acc  = []
time = []

for i in range(N):
    gc.collect()
    start = timeit.default_timer()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_pil = img
    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for c in annots:
        
        img_cam = camnet.get_top(img_pil, c)
        img_cam = cam.cam_process(img_pil, img_cam)
        pred = (cam.cam_to_gcmask(img_cam, a = 0.0, b = FB_threshold, c = 1.0) % 2).astype(bool)
        
        true = voc.true_mask(sgm, classes.index(c) + 1).astype(bool)
        
        IoU.append(voc.IoU(true, pred, sgm))
        Acc.append(voc.accuracy(true, pred, sgm))
        
    stop = timeit.default_timer()
    time.append(stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time), shift = 2))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i), shift = 2))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print(f'Mean Acc                 =  {np.mean(Acc) :.3f}')
        print()

Image nb 0
Time spent               =  0m00s
Estimated time remaining =  4m47s
Mean IoU                 =  0.068
Mean Acc                 =  0.975

Image nb 20
Time spent               =  0m01s
Estimated time remaining =  2m53s
Mean IoU                 =  0.037
Mean Acc                 =  0.761

Image nb 40
Time spent               =  0m02s
Estimated time remaining =  3m07s
Mean IoU                 =  0.023
Mean Acc                 =  0.819

Image nb 60
Time spent               =  0m04s
Estimated time remaining =  3m10s
Mean IoU                 =  0.022
Mean Acc                 =  0.831

Image nb 80
Time spent               =  0m05s
Estimated time remaining =  3m09s
Mean IoU                 =  0.023
Mean Acc                 =  0.845

Image nb 100
Time spent               =  0m06s
Estimated time remaining =  3m07s
Mean IoU                 =  0.021
Mean Acc                 =  0.844

Image nb 120
Time spent               =  0m07s
Estimated time remaining =  3m04s
Mean IoU                 

Image nb 1100
Time spent               =  1m02s
Estimated time remaining =  1m42s
Mean IoU                 =  0.018
Mean Acc                 =  0.828

Image nb 1120
Time spent               =  1m03s
Estimated time remaining =  1m40s
Mean IoU                 =  0.018
Mean Acc                 =  0.827

Image nb 1140
Time spent               =  1m04s
Estimated time remaining =  1m39s
Mean IoU                 =  0.018
Mean Acc                 =  0.826

Image nb 1160
Time spent               =  1m05s
Estimated time remaining =  1m38s
Mean IoU                 =  0.018
Mean Acc                 =  0.826

Image nb 1180
Time spent               =  1m06s
Estimated time remaining =  1m37s
Mean IoU                 =  0.018
Mean Acc                 =  0.825

Image nb 1200
Time spent               =  1m07s
Estimated time remaining =  1m35s
Mean IoU                 =  0.018
Mean Acc                 =  0.825

Image nb 1220
Time spent               =  1m08s
Estimated time remaining =  1m34s
Mean IoU    

Image nb 2200
Time spent               =  2m00s
Estimated time remaining =  0m39s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2220
Time spent               =  2m01s
Estimated time remaining =  0m37s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2240
Time spent               =  2m02s
Estimated time remaining =  0m36s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2260
Time spent               =  2m04s
Estimated time remaining =  0m35s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2280
Time spent               =  2m05s
Estimated time remaining =  0m34s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2300
Time spent               =  2m06s
Estimated time remaining =  0m33s
Mean IoU                 =  0.019
Mean Acc                 =  0.821

Image nb 2320
Time spent               =  2m07s
Estimated time remaining =  0m32s
Mean IoU    

In [10]:
print(f'Mean IoU = {np.mean(IoU):.3f}')
print(f'Mean Acc = {np.mean(Acc):.3f}')

Mean IoU = 0.019
Mean Acc = 0.818


In [11]:
df = pd.read_csv(json_path + 'voc-class-annotations.csv')

In [12]:
df['IoU'] = IoU
df['Accuracy'] = Acc

In [13]:
df.to_csv(output_path + 'IoU.csv', index = False)